In [2]:
import numpy as np

from litpose_app.deps import project_info
from litpose_app.routes.rglob import rglob, RGlobRequest
from multiview_autolabel import *
from bundle_adjust import *
warm_up_anipose()

In [10]:
import numpy as np
# Assuming Camera and CameraGroup are in a file named 'cameras.py'
from aniposelib.cameras import CameraGroup, Camera


def demonstrate_triangulate():
    # --- 1. Create dummy Camera objects ---
    # For demonstration, we'll create two cameras with simple parameters.
    # In a real scenario, these would come from calibration.

    # Camera 1 parameters
    camera_matrix_1 = np.array([[1000, 0, 640],
                                [0, 1000, 480],
                                [0, 0, 1]], dtype=np.float64)
    dist_coeffs_1 = np.array([0, 0, 0, 0, 0], dtype=np.float64)  # No distortion for simplicity
    rvec_1 = np.array([[0.], [0.], [0.]], dtype=np.float64)  # No rotation
    tvec_1 = np.array([[0.], [0.], [0.]], dtype=np.float64)  # Origin at world origin
    cam1 = Camera(name='cam1',
                  matrix=camera_matrix_1,
                  dist=dist_coeffs_1,
                  rvec=rvec_1,
                  tvec=tvec_1,
                  size=(1280, 960))  # Image width, height

    # Camera 2 parameters - shifted and slightly rotated relative to Camera 1
    camera_matrix_2 = np.array([[1050, 0, 650],
                                [0, 1050, 470],
                                [0, 0, 1]], dtype=np.float64)
    dist_coeffs_2 = np.array([0.01, -0.02, 0, 0, 0], dtype=np.float64)  # Some distortion
    rvec_2 = np.array([[0.05], [-0.1], [0.03]], dtype=np.float64)  # Small rotation
    tvec_2 = np.array([[0.5], [0.1], [-0.2]], dtype=np.float64)  # Translated from origin
    cam2 = Camera(name='cam2',
                  matrix=camera_matrix_2,
                  dist=dist_coeffs_2,
                  rvec=rvec_2,
                  tvec=tvec_2,
                  size=(1280, 960))

    print("Cameras initialized.")
    print(f"Camera 1 name: {cam1.get_name()}")
    print(f"Camera 2 name: {cam2.get_name()}")

    # --- 2. Create a CameraGroup ---
    camera_group = CameraGroup(cameras=[cam1, cam2])
    print("\nCameraGroup created with 2 cameras.")

    # --- 3. Define a sample 3D point (world coordinates) ---
    # We will try to triangulate this point
    true_3d_point = np.array([[1.0, 2.0, 5.0]], dtype=np.float64)  # Shape (1, 3)

    print(f"\nTrue 3D point in world coordinates: {true_3d_point}")

    # --- 4. Project the 3D point onto each camera to get 2D observations ---
    # The 'project' method of CameraGroup expects points for each camera.
    # The output of Camera.project is (N, 2) for N points.
    points_2d_cam1 = cam1.project(true_3d_point)
    points_2d_cam2 = cam2.project(true_3d_point)

    # Reshape the points to match expected format (num_cameras, num_points, 2)
    points_2d_cam1 = points_2d_cam1.reshape(1, 2)
    points_2d_cam2 = points_2d_cam2.reshape(1, 2)

    # TEST NAN
    #points_2d_cam1[0, 0] = np.nan

    points_for_triangulation = np.stack([points_2d_cam1, points_2d_cam2])

    print(f"\nProjected 2D point for Camera 1:\n{points_2d_cam1}")
    print(f"Projected 2D point for Camera 2:\n{points_2d_cam2}")
    print(f"Shape of points for triangulation: {points_for_triangulation.shape}")

    # --- 5. Use CameraGroup.triangulate to reconstruct the 3D point ---
    # We'll set undistort=True because cam2 has distortion coefficients.
    # The triangulate method returns a numpy array of shape (num_points, 3).
    triangulated_points = camera_group.triangulate(points_for_triangulation, undistort=True)

    print(f"\nTriangulated 3D points:\n{triangulated_points}")

    # Compare the triangulated point with the original 3D point
    if triangulated_points.shape[0] > 0:
        reconstructed_point = triangulated_points[0]
        error = np.linalg.norm(reconstructed_point - true_3d_point[0])
        print(f"\nOriginal 3D point: {true_3d_point[0]}")
        print(f"Reconstructed 3D point: {reconstructed_point}")
        print(f"Reconstruction error (L2 norm): {error:.6f}")


Cameras initialized.
Camera 1 name: cam1
Camera 2 name: cam2

CameraGroup created with 2 cameras.

True 3D point in world coordinates: [[1. 2. 5.]]

Projected 2D point for Camera 1:
[[ nan 880.]]
Projected 2D point for Camera 2:
[[847.82290042 865.22061803]]
Shape of points for triangulation: (2, 1, 2)

Triangulated 3D points:
[[nan nan nan]]

Original 3D point: [1. 2. 5.]
Reconstructed 3D point: [nan nan nan]
Reconstruction error (L2 norm): nan


In [3]:

from collections import defaultdict
from pathlib import Path

request = BundleAdjustRequest(sessionKey='05272019_fly1_0_R1C24__rot-ccw-0.06_sec')

from litpose_app import deps
config = deps.config()
project_info = deps.project_info(config)

bundle_adjust(request, project_info, config)




ValidationError: 1 validation error for BundleAdjustRequest
mvlabelfile
  Field required [type=missing, input_value={'sessionKey': '05272019_...1C24__rot-ccw-0.06_sec'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/missing